# Setup

In [2]:
%matplotlib inline

import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from torch.autograd import Variable
from torchvision import datasets, transforms

sns.set()

In [ ]:
torch.nn.

# Hyperparameter space

In [3]:
LAYER_SPACE = {
    'nb_units':{'lb': 128, 'ub':1024, 'mutate': 0.15},
    'dropout_rate': {'lb': 0.0, 'ub': 0.7, 'mutate': 0.2},
    'activation': {'func': ['linear','tanh','relu','sigmoid','elu'], 'mutate':0.2}
}

In [4]:
NET_SPACE = {
    'nb_layers': {'lb': 1, 'ub': 3, 'mutate': 0.15},
    'lr': {'lb': 0.001, 'ub':0.1, 'mutate': 0.15},
    'weight_decay': {'lb': 0.00001, 'ub': 0.0004, 'mutate':0.2},
    'optimizer': {'func': ['sgd', 'adam', 'adadelta','rmsprop'], 'mutate': 0.2}
}

# Randomise network

In [5]:
def random_value(space):
    """Returns random value from space."""
    
    val = None
    
    if 'func' in space: #randomise optimiser or activation function
        val = random.sample(space['func'], 1)[0] 
    
    elif isinstance(space['lb'], int): #randomise number of units or layers
        val = random.randint(space['lb'], space['ub'])
    
    else: #randomise percentages, i.e. dropout rates or weight decay
        val = random.random() * (space['ub'] - space['lb']) + space['lb']
    
    return val

In [6]:
def randomize_network(layer_space, net_space): 
    """Returns a randomised neural network"""
    net = {}
    
    for key in net_space.keys():
        net[key] = random_value(net_space[key])
        
    layers = []
    
    for i in range(net['nb_layers']):
        layer = {}
        for key in layer_space.keys():
            layer[key] = random_value(layer_space[key])
        layers.append(layer)
        net['layers'] = layers
        
    return net

In [7]:
randomize_network(LAYER_SPACE, NET_SPACE)

{'layers': [{'activation': 'tanh',
   'dropout_rate': 0.055676525173730444,
   'nb_units': 132},
  {'activation': 'relu', 'dropout_rate': 0.4407416076783757, 'nb_units': 668},
  {'activation': 'elu', 'dropout_rate': 0.3728920305155153, 'nb_units': 982}],
 'lr': 0.0854667753745106,
 'nb_layers': 3,
 'optimizer': 'adam',
 'weight_decay': 9.064557628116618e-05}

# Mutate network

In [42]:
def mutate_net(net, layer_space, net_space):
    
    # mutate optimizer
    for k in ['lr', 'weight_decay', 'optimizer']:
        if random.random() < net_space[k]['mutate']:
            net[k] = random_value(net_space[k])
    
    
    # mutate layers
    for layer in net['layers']:
        for k in layer_space.keys():
            if random.random() < layer_space[k]['mutate']:
                layer[k] = random_value(layer_space[k])
                
                
    # mutate number of layers -- 50% add 50% remove
    if random.random() < net_space['nb_layers']['mutate']:
        if net['nb_layers'] <= net_space['nb_layers']['ub']:
            if random.random()< 0.5 and \
            net['nb_layers'] < net_space['nb_layers']['ub']:
                layer = {}
                for key in layer_space.keys():
                    layer[key] = random_value(layer_space[key])
                net['layers'].append(layer)      
            else:
                if net['nb_layers'] > 1:
                    net['layers'].pop()

                
            # value & id update
            net['nb_layers'] = len(net['layers'])         
            
    return net

# NetBuilder

In [84]:
net

{'layers': [{'activation': 'sigmoid',
   'dropout_rate': 0.20358944402212859,
   'nb_units': 877},
  {'activation': 'linear',
   'dropout_rate': 0.43393763424723497,
   'nb_units': 981}],
 'lr': 0.00757973998441296,
 'nb_layers': 2,
 'optimizer': 'adam',
 'weight_decay': 0.000123059009610768}

In [77]:
class Flatten(nn.Module):
    """For use in NetFromGenome."""

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)

In [125]:
class NetFromBuildInfo(nn.Module):

    def __init__(self, build_info):
        super(NetFromBuildInfo, self).__init__()
        
        self.activation_dict = {
            'tanh': nn.Tanh(),
            'relu': nn.ReLU(),
            'sigmoid': nn.Sigmoid(),
            'elu': nn.ELU()
            }
        
        previous_units = 28 * 28 #MNIST shape
        
        self.model = nn.Sequential()
        self.model.add_module('flatten', Flatten())
         
        for i, layer_info in enumerate(build_info['layers']):
            
            i = str(i)
            
            self.model.add_module(
                'fc_' + i,
                nn.Linear(previous_units, layer_info['nb_units'])
                )
            self.model.add_module(
                'dropout_' + i,
                nn.Dropout(p=layer_info['dropout_rate'])
                )
            
            if layer_info['activation'] == 'linear':
                continue #linear activation is identity function
            self.model.add_module(
                layer_info['activation']+ i,
                self.activation_dict[layer_info['activation']])
            
            previous_units = layer_info['nb_units']

        self.model.add_module(
            'logits',
            nn.Linear(previous_units, 10) #10 MNIST classes
            )
        
        self.model.add_module('logsoftmax', nn.LogSoftmax())
        self.model.cuda()
        

        self.opt_args = {#'params': self.model.parameters(),
                 'weight_decay': build_info['weight_decay'],
                 'lr': build_info['lr']
                 }
        
        self.optimizer_dict = {
            'adam': optim.Adam(self.model.parameters(),**self.opt_args),
            'rmsprop': optim.RMSprop(self.model.parameters(),**self.opt_args),
            'adadelta':optim.Adadelta(self.model.parameters(),**self.opt_args),
            'sgd': optim.SGD(self.model.parameters(), **self.opt_args, momentum=0.9) #momentum to train faster
            }
        
        print(self.opt_args)

        self.optimizer = self.optimizer_dict[build_info['optimizer']]

In [126]:
test = NetFromBuildInfo(net)

{'weight_decay': 0.000123059009610768, 'lr': 0.00757973998441296}


In [120]:
net

{'layers': [{'activation': 'sigmoid',
   'dropout_rate': 0.20358944402212859,
   'nb_units': 877},
  {'activation': 'linear',
   'dropout_rate': 0.43393763424723497,
   'nb_units': 981}],
 'lr': 0.00757973998441296,
 'nb_layers': 2,
 'optimizer': 'adam',
 'weight_decay': 0.000123059009610768}

In [127]:
test

NetFromBuildInfo(
  (model): Sequential(
    (flatten): Flatten()
    (fc_0): Linear(in_features=784, out_features=877, bias=True)
    (dropout_0): Dropout(p=0.20358944402212859)
    (sigmoid0): Sigmoid()
    (fc_1): Linear(in_features=877, out_features=981, bias=True)
    (dropout_1): Dropout(p=0.43393763424723497)
    (logits): Linear(in_features=877, out_features=10, bias=True)
    (logsoftmax): LogSoftmax()
  )
)

In [132]:
mnist_test = np.random.rand(1, 1, 28, 28)

In [139]:
mnist_test = torch.from_numpy(mnist_test)

In [141]:
test(mnist_test)

NotImplementedError: 